In [ ]:
import os
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import MinMaxScaler

# 1. Read Excel data
file_path = "merged_output_cleaned.xlsx"
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Error: File not found at {file_path}")

df = pd.read_excel(file_path, engine="openpyxl")

# 2. Clean the 'price' column
# Remove non-numeric characters and convert to float
df["price"] = (
    df["price"]
    .astype(str)
    .str.replace("€", "", regex=False)
    .str.replace("/ month", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+\.?\d*)")  # Keep only numbers
    .astype(float)
)
# Filter out invalid price data
df = df.dropna(subset=["price"])

# 3. Fix image paths
# Remove prefixes like 'property_images22/'
df["image"] = df["image"].str.replace("property_images22\\", "", regex=False)
df["image"] = df["image"].str.replace("property_images22/", "", regex=False)

# Define the image folder
image_folder = "merged_images"

def extract_features(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = model.predict(img_array)
        return features.flatten()
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

# 4. Load MobileNetV2 for feature extraction
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
model = Model(inputs=base_model.input, outputs=base_model.output)

image_features = []
image_files = []

# 5. Extract image features
for img_name in df["image"]:
    img_path = os.path.join(image_folder, img_name)
    if os.path.exists(img_path):
        features = extract_features(img_path)
        if features is not None:
            image_features.append(features)
            image_files.append(img_name)
    else:
        print(f"Warning: Image file {img_path} not found!")

# Convert to NumPy array
image_features = np.array(image_features)
print(f"Extracted image feature shape: {image_features.shape}")

if image_features.shape[0] == 0:
    raise ValueError("Error: No valid image features extracted. Check image paths and extraction process.")

# 6. Perform dimensionality reduction using UMAP
umap_model = umap.UMAP(n_components=2, n_neighbors=30, min_dist=0.3, random_state=42)
X_umap = umap_model.fit_transform(image_features)

# 7. Normalize price data
df_filtered = df[df["image"].isin(image_files)]
price_data = df_filtered["price"].values.reshape(-1, 1)
scaler = MinMaxScaler()
price_scaled = scaler.fit_transform(price_data).flatten()

# 8. Visualization
plt.figure(figsize=(10, 6))
plt.scatter(X_umap[:, 0], X_umap[:, 1], c=price_scaled, cmap="coolwarm", alpha=0.7)
plt.colorbar(label="Price (Normalized)")
plt.title("Price Analysis of Airbnb Listings Based on Image Features")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")

# Annotate the top 10 highest price points
top_10_idx = np.argsort(-price_scaled)[:10]  # Get the indices of the top 10 highest prices
for idx in top_10_idx:
    plt.annotate(f"{df_filtered.iloc[idx]['price']:.0f}", (X_umap[idx, 0], X_umap[idx, 1]),
                 textcoords="offset points", xytext=(5,5), ha='right', fontsize=9, color='black', weight='bold')

plt.show()


C:\Users\lenovo\AppData\Local\Temp\ipykernel_19864\456710754.py:54: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154m

D:\Anaconda\envs\RC11SKILL\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121